In [1]:
# change the text here
clip_text = ["a man sits back down on the ground"]


import sys
sys.argv = ['GPT_eval_multi.py']
import options.option_transformer as option_trans
args = option_trans.get_args_parser()

args.dataname = 't2m'
args.resume_pth = 'VQVAE/net_last.pth'
args.resume_trans = 'VQTransformer_corruption05/net_best_fid.pth'
args.down_t = 2
args.depth = 3
args.block_size = 51
import clip
import torch
import numpy as np
import models.vqvae as vqvae
import models.t2m_trans as trans
import warnings
warnings.filterwarnings('ignore')

## load clip model and datasets
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False, download_root='./')  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate)


trans_encoder = trans.Text2Motion_Transformer(num_vq=args.nb_code,
                                embed_dim=1024,
                                clip_dim=args.clip_dim,
                                block_size=args.block_size,
                                num_layers=9,
                                n_head=16,
                                drop_out_rate=args.drop_out_rate,
                                fc_rate=args.ff_rate)


print ('loading checkpoint from {}'.format(args.resume_pth))
ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
net.eval()
net.cuda()

print ('loading transformer checkpoint from {}'.format(args.resume_trans))
ckpt = torch.load(args.resume_trans, map_location='cpu')
trans_encoder.load_state_dict(ckpt['trans'], strict=True)
trans_encoder.eval()
trans_encoder.cuda()

mean = torch.from_numpy(np.load('./checkpoints/t2m/VQVAEV3_CB1024_CMT_H1024_NRES3/meta/mean.npy')).cuda()
std = torch.from_numpy(np.load('./checkpoints/t2m/VQVAEV3_CB1024_CMT_H1024_NRES3/meta/std.npy')).cuda()

text = clip.tokenize(clip_text, truncate=True).cuda()
feat_clip_text = clip_model.encode_text(text).float()
index_motion = trans_encoder.sample(feat_clip_text[0:1], False)
print(index_motion)
pred_pose = net.forward_decoder(index_motion)

from utils.motion_process import recover_from_ric
pred_xyz = recover_from_ric((pred_pose*std+mean).float(), 22)
xyz = pred_xyz.reshape(1, -1, 22, 3)
#
np.save('/mnt/disk_2/jinpeng/t2m-gpt/0426/motion.npy', pred_xyz.detach().cpu().numpy())
#
import visualization.plot_3d_global as plot_3d
pose_vis = plot_3d.draw_to_batch(xyz.detach().cpu().numpy(),clip_text, ['example_long.gif'])

/home/jinpeng/miniconda3/envs/humanise/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


loading checkpoint from VQVAE/net_last.pth
loading transformer checkpoint from VQTransformer_corruption05/net_best_fid.pth
tensor([[326, 256, 233, 189, 189, 296,  79, 357,  17, 449, 449, 205, 449, 449,
         449, 286, 449, 449, 255,  33, 112, 344, 439, 510,  12, 510, 421, 301]],
       device='cuda:0')


In [5]:
a = torch.IntTensor([472, 472, 472, 472, 472, 472, 230,  45,  45, 134, 134,  34,  34, 176,
         313, 383, 441, 329, 102, 383, 104, 494, 159, 104, 104, 104, 104, 326, 256, 233, 189, 189, 296,  79, 357,  17, 449, 449, 205, 449, 449,
         449, 286, 449, 449, 255,  33, 112, 344, 439, 510,  12, 510, 421, 301])
a = a.cuda()
b = a.reshape(-1, 1)
b = b.cuda()


In [6]:
from utils.motion_process import recover_from_ric
import visualization.plot_3d_global as plot_3d
pred_pose = net.forward_decoder(b)
pred_xyz = recover_from_ric((pred_pose*std+mean).float(), 22)
xyz = pred_xyz.reshape(1, -1, 22, 3)
pose_vis = plot_3d.draw_to_batch(xyz.detach().cpu().numpy(),str(b), ['example_fail7.gif'])

In [104]:
pred_pose = net.forward_decoder(b)
pred_pose.shape
# pred_xyz = recover_from_ric((pred_pose*std+mean).float(), 22)
# pred_xyz[0][0]

tensor([[ 16],
        [423]], device='cuda:0', dtype=torch.int32)


torch.Size([1, 8, 263])

In [101]:
pred_pose = net.forward_decoder(b)
pred_pose[0][0]
# pred_xyz = recover_from_ric((pred_pose*std+mean).float(), 22)
# pred_xyz[0][0]

tensor([[16]], device='cuda:0', dtype=torch.int32)


tensor([ 1.6720e-01,  1.8258e-01, -8.1907e+00,  2.8584e+00,  2.0302e-02,
         1.5824e-01,  3.7372e-02, -2.9800e-02,  1.4712e-01,  7.2126e-02,
        -1.6510e-02,  1.4945e-01,  4.0792e-02,  2.8900e-02,  6.5042e-02,
        -8.0310e-02,  1.3124e-02,  3.6451e-02, -3.3398e-02, -2.0505e-04,
         2.1382e-01, -7.2045e-02,  2.3530e-02, -7.8821e-02,  9.3246e-02,
        -4.7077e-02, -1.1725e-01,  9.1720e-02,  5.1977e-03,  2.0888e-01,
        -6.8083e-02, -4.0574e-03, -1.4361e-01, -3.0812e-02,  9.2329e-02,
        -1.6321e-01, -1.3288e-02, -3.4276e-02,  2.8288e-01, -1.9559e-01,
        -6.2485e-03,  2.5408e-01, -1.3930e-01, -2.7731e-02,  2.2538e-01,
        -1.5520e-01, -5.3694e-03,  3.3805e-01, -2.7618e-01,  3.4632e-02,
         2.7135e-01, -1.2450e-01, -7.7526e-02,  2.1407e-01, -1.6245e-01,
         5.2300e-02,  6.8070e-02, -1.1350e-01, -2.6924e-02,  7.0502e-03,
        -2.2611e-01,  9.9313e-02, -2.1845e-01, -1.4740e-01, -1.5291e-01,
        -3.8564e-01, -3.2547e-01,  4.3136e-01,  4.7

In [114]:
for number1 in b:
    # for number in number1:
    print(number1)
    # clip_text = ["183"]
    pred_pose = net.forward_decoder(number1)
    pred_xyz = recover_from_ric((pred_pose*std+mean).float(), 22)
    xyz = pred_xyz.reshape(1, -1, 22, 3)
    # np.save('/mnt/disk_2/jinpeng/t2m-gpt/0426/fail/'+str(number.cpu().numpy())+'.npy', pred_xyz.detach().cpu().numpy())
    pose_vis = plot_3d.draw_to_batch(xyz.detach().cpu().numpy(),str(number1), ['/mnt/disk_2/jinpeng/t2m-gpt/0426/fail3/'+str(number1.cpu().numpy()) + '.gif'])

tensor([284], device='cuda:0', dtype=torch.int32)
tensor([284], device='cuda:0', dtype=torch.int32)
tensor([387], device='cuda:0', dtype=torch.int32)
tensor([387], device='cuda:0', dtype=torch.int32)
tensor([387], device='cuda:0', dtype=torch.int32)
tensor([387], device='cuda:0', dtype=torch.int32)
tensor([457], device='cuda:0', dtype=torch.int32)
tensor([457], device='cuda:0', dtype=torch.int32)
tensor([6], device='cuda:0', dtype=torch.int32)
tensor([6], device='cuda:0', dtype=torch.int32)
tensor([407], device='cuda:0', dtype=torch.int32)
tensor([407], device='cuda:0', dtype=torch.int32)
tensor([44], device='cuda:0', dtype=torch.int32)
tensor([44], device='cuda:0', dtype=torch.int32)


In [ ]:
pred_pose.shape

In [ ]:
from utils.motion_process import recover_from_ric
pred_pose = net.forward_decoder(b)
pred_pose.shape
pred_xyz = recover_from_ric((pred_pose*std+mean).float(), 22)
xyz = pred_xyz.reshape(1, -1, 22, 3)
np.save('/mnt/disk_2/jinpeng/t2m-gpt/0426/fail8.npy', pred_xyz.detach().cpu().numpy())

In [ ]:
pose_vis = plot_3d.draw_to_batch(xyz.detach().cpu().numpy(),clip_text, ['example_fail7.gif'])

In [ ]:
pred_xyz.shape

In [ ]:
import sys

original = '/mnt/disk_2/jinpeng/t2m-gpt/visualize/joints2smpl/smpl_models/gmm_08.pkl'
destination = "/mnt/disk_2/jinpeng/t2m-gpt/visualize/joints2smpl/smpl_models/gmm_08_unix.pkl"

content = ''
outsize = 0
with open(original, 'rb') as infile:
    content = infile.read()
with open(destination, 'wb') as output:
    for line in content.splitlines():
        outsize += len(line) + 1
        output.write(line + str.encode('\n'))

print("Done. Saved %s bytes." % (len(content)-outsize))